In [16]:
import pyxdf
import mne
import numpy as np
import pandas as pd
streams, header = pyxdf.load_xdf("C:/Users/Toey/Desktop/data_ssvep/Toey/exp1/block_1.xdf")

In [17]:
# สร้าง list เพื่อเก็บ index ของ epoch ที่ต้องการลบ event_id ออก
def count_each_class(mne_epochs, num_class):
    indices_to_drop = []
    num_class = num_class

    # Iterate ผ่านทุกๆ epoch
    for idx, epoch in enumerate(mne_epochs):
        # ตรวจสอบว่า event_id เป็น {'natural': 0} หรือไม่
        if mne_epochs.events[idx, -1] == num_class:  # ตรวจสอบ event_id ในข้อมูล events ของ epoch นั้นๆ
            # เพิ่ม index ของ epoch ที่ต้องการลบออกใน list
            indices_to_drop.append(idx)

    # ลบ epoch ที่มี event_id เป็น {'natural': 0} ออกจาก mne_epochs
    return indices_to_drop
    # mne_epochs.drop(indices_to_drop)

In [18]:
import random

def random_drop_to_balance(mne_epochs, class_counts):
    # หาจำนวนของ class ที่มีจำนวนน้อยสุด
    # print(class_counts)
    min_count = min(class_counts)
    
    # สร้าง list เพื่อเก็บ index ของแต่ละ class
    class_indices = [[] for _ in range(len(class_counts))]

    # นับ index ของแต่ละ class
    for idx, epoch in enumerate(mne_epochs):
        event_id = mne_epochs.events[idx, -1]
        class_indices[event_id].append(idx)

    # สุ่มลำดับ index ของแต่ละ class เพื่อทำการลบ
    random_indices_to_drop = []
    for indices in class_indices:
        # สุ่มลำดับ index ที่ต้องการลบออก
        if len(indices) > len(min_count):
            random_indices = random.sample(indices, len(indices) - len(min_count))
            random_indices_to_drop.extend(random_indices)
    
    # ลบ epoch ที่ต้องการออกจาก mne_epochs
    mne_epochs.drop(random_indices_to_drop)

    return random_indices_to_drop

In [19]:
raw_data = streams[0]["time_series"].T #From Steam variable this query is EEG data
raw_data.shape # It contain with 8 channel of EGG data

(8, 331950)

In [20]:
channels =['Fz','Cz','P4','Pz','P3','P08','Oz','P07'] #Set your target EEG channel name
info = mne.create_info(
    ch_names= channels,
    ch_types= ['eeg']*len(channels),
    sfreq= 250 #OpenBCI Frequency acquistion
)
# Create MNE rawarray
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)

In [21]:
raw_mne = mne.io.RawArray(raw_data, info, verbose=False)
print(raw_mne)
event_index = streams[1]["time_series"].T[0] #Get all event marker of experiment 
#.T คือ transpose array [0] data ใน column 0 ของ streams[1]["time_series"]
print(event_index.shape)
event_timestamp = streams[1]["time_stamps"].T #Timestamp when event marked
# รับเวลาที่ marker นั้นอยู่
print(event_timestamp.shape)

<RawArray | 8 x 331950 (1327.8 s), ~20.3 MB, data loaded>
(100,)
(100,)


In [22]:
events = np.column_stack((np.array(event_timestamp, dtype = int),
                        np.zeros(len(event_timestamp), dtype = int),
                        np.array(event_index, dtype = int)))
fre1 = 8
fre2 = 10
fre3 = 12
fre4 = 14
events_id = { # Set up your event name
     f'natural': 0,
     f'Frequency {fre1} Hz': 1,
     f'Frequency {fre2} Hz': 2,
     f'Frequency {fre3} Hz': 3,
     f'Frequency {fre4} Hz': 4,
}
mne_epochs = mne.Epochs(raw_mne, events, 
        tmin= -0.5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax= 20,    # final timestamp (10 means set epoch duration 10 second)
        event_id =events_id,
        preload = True,
        event_repeated='drop',
        baseline=(-0.5, 0)
    )
mne_epochs

Not setting metadata
100 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 5126 original time points ...
0 bad epochs dropped


Number of events,100
Events,Frequency 10 Hz: 11Frequency 12 Hz: 11Frequency 14 Hz: 11Frequency 8 Hz: 11natural: 56
Time range,-0.500 – 20.000 s
Baseline,-0.500 – 0.000 s


In [23]:
mne_epochs.get_data().shape

C:\Users\Toey\AppData\Local\Temp\ipykernel_348\1290654083.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  mne_epochs.get_data().shape


(100, 8, 5126)

In [24]:
class0 = count_each_class(mne_epochs, 0)
class1 = count_each_class(mne_epochs, 1)
class2 = count_each_class(mne_epochs, 2)
class3 = count_each_class(mne_epochs, 3)
class4 = count_each_class(mne_epochs, 4)

class_counts = [class0, class1, class2, class3, class4]
random_indices_to_drop = random_drop_to_balance(mne_epochs, class_counts)

Dropped 45 epochs: 3, 5, 7, 8, 10, 12, 14, 16, 17, 19, 21, 23, 25, 26, 28, 30, 32, 35, 37, 39, 41, 44, 46, 48, 50, 53, 55, 57, 59, 61, 62, 66, 68, 70, 73, 77, 79, 80, 86, 88, 91, 93, 95, 98, 99


In [25]:
mne_epochs.get_data().shape

C:\Users\Toey\AppData\Local\Temp\ipykernel_348\1290654083.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  mne_epochs.get_data().shape


(55, 8, 5126)

In [26]:
mne_epochs

Number of events,55
Events,Frequency 10 Hz: 11Frequency 12 Hz: 11Frequency 14 Hz: 11Frequency 8 Hz: 11natural: 11
Time range,-0.500 – 20.000 s
Baseline,-0.500 – 0.000 s


In [27]:
x = mne_epochs.get_data()
y = mne_epochs.events[:, -1]

# print(x.shape)
# print(y.shape)

(55, 8, 5126)
(55,)


C:\Users\Toey\AppData\Local\Temp\ipykernel_348\1471954283.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  x = mne_epochs.get_data()
